In [5]:
import os
import time
import requests
import sys

In [7]:
def retrieve_html():
    for year in range(2013,2022):
        for month in range(1,13):
            if (month<10):
                url="http://en.tutiempo.net/climate/0{}-{}/ws-429710.html".format(month,year)
        else:
            url="http://en.tutiempo.net/climate/{}-{}/ws-429710.html".format(month,year)
            texts=requests.get(url)
            text_utf=texts.text.encode('utf=8')
            
            if not os.path.exists("Data_Set1/Html_Data_Set1/{}".format(year)):
                os.makedirs("Data_Set1/Html_Data_Set1/{}".format(year))
            with open("Data_Set1/Html_Data_Set1/{}/{}.html".format(year,month),"wb") as output:
                output.write(text_utf)
            
        sys.stdout.flush()
        
if __name__=="__main__":
 start_time=time.time()
 retrieve_html()
 stop_time=time.time()
 print("Time taken {}".format(stop_time-start_time))
        

Time taken 19.126678705215454


In [3]:
import requests
import sys
import pandas as pd
from bs4 import BeautifulSoup
import os
import csv

In [9]:
def met_data(month, year):

    file_html = open('Data_Set1/Html_Data_Set1/{}/{}.html'.format(year,month), 'rb')
    plain_text = file_html.read()

    tempD = []
    finalD = []

    soup = BeautifulSoup(plain_text, "lxml")
    for table in soup.findAll('table', {'class': 'medias mensuales numspan'}):
        for tbody in table:
            for tr in tbody:
                a = tr.get_text()
                tempD.append(a)

    rows = len(tempD) / 15

    for times in range(round(rows)):
        newtempD = []
        for i in range(15):
            newtempD.append(tempD[0])
            tempD.pop(0)
        finalD.append(newtempD)

    length = len(finalD)

    finalD.pop(length - 1)
    finalD.pop(0)

    for a in range(len(finalD)):
        finalD[a].pop(6)
        finalD[a].pop(13)
        finalD[a].pop(12)
        finalD[a].pop(11)
        finalD[a].pop(10)
        finalD[a].pop(9)
        finalD[a].pop(0)

    return finalD

def data_combine(year, cs):
    for a in pd.read_csv('Data_Set1/Real-Data_Set1/real_' + str(year) + '.csv', chunksize=cs):
        df = pd.DataFrame(data=a)
        mylist = df.values.tolist()
    return mylist


if __name__ == "__main__":
    if not os.path.exists("Data_Set1/Real-Data_Set1"):
        os.makedirs("Data_Set1/Real-Data_Set1")
    for year in range(2013, 2017):
        final_data = []
        with open('file:///C:/Users/DELL/Data_Set1/Html_Data_Set1/2013/12.html' + str(year) + '.csv', 'w') as csvfile:
            wr = csv.writer(csvfile, dialect='excel')
            wr.writerow(
                ['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM 2.5'])
        for month in range(1, 13):
            temp = met_data(month, year)
            final_data = final_data + temp

        pm = getattr(sys.modules[__name__], 'avg_data_{}'.format(year))()

        if len(pm) == 364:
            pm.insert(364, '-')

        for i in range(len(final_data)-1):
            # final[i].insert(0, i + 1)
            final_data[i].insert(8, pm[i])

        with open('Data_Set1/Real-Data_Set1/real_' + str(year) + '.csv', 'a') as csvfile:
            wr = csv.writer(csvfile, dialect='excel')
            for row in final_data:
                flag = 0
                for elem in row:
                    if elem == "" or elem == "-":
                        flag = 1
                if flag != 1:
                    wr.writerow(row)

    data_2013 = data_combine(2013, 600)
    data_2014 = data_combine(2014, 600)
    data_2015 = data_combine(2015, 600)
    data_2016 = data_combine(2016, 600)

    total=data_2013+data_2014+data_2015+data_2016

    with open('Data_Set1/Real-Data_Set1/Real_Combine.csv', 'w') as csvfile:
        wr = csv.writer(csvfile, dialect='excel')
        wr.writerow(
            ['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM 2.5'])
        wr.writerows(total)


df=pd.read_csv('Data_Set1/Real-Data_Set1/AQI_Combine.csv')

OSError: [Errno 22] Invalid argument: 'file:///C:/Users/DELL/Data_Set1/Html_Data_Set1/2013/12.html2013.csv'